In [2]:
import dotenv
import json
import os
import requests
import sys
from rich.progress import track
from concurrent.futures import ThreadPoolExecutor, as_completed

import censusgeocode

from census import Census
from us import states

dotenv.load_dotenv()


CURRENT_DIR = %pwd
ROOT_DIR = os.path.dirname(CURRENT_DIR)

SRC_DIR = os.path.join(ROOT_DIR, 'src')
sys.path.append(SRC_DIR)

CENSUS_API_KEY = os.environ["CENSUS_API_KEY"]
GOOGLE_API_KEY = os.environ["GOOGLE_MAPS_PLACES_API_KEY"]


In [3]:
from search_context.us_census import USCensusAPI
from search_context.google_places import GooglePlacesAPI

In [4]:
c = Census(CENSUS_API_KEY)
c.acs5.get(('NAME', 'B25034_010E'),
          {'for': 'state:{}'.format(states.MD.fips)})

[{'NAME': 'Maryland', 'B25034_010E': 130857.0, 'state': '24'}]

In [4]:
tables = c.acs5.tables()

In [5]:
tables

[{'name': 'B17015',
  'description': 'POVERTY STATUS IN THE PAST 12 MONTHS OF FAMILIES BY FAMILY TYPE BY SOCIAL SECURITY INCOME BY SUPPLEMENTAL SECURITY INCOME (SSI) AND CASH PUBLIC ASSISTANCE INCOME',
  'variables': 'http://api.census.gov/data/2021/acs/acs5/groups/B17015.json',
  'universe ': 'Families'},
 {'name': 'B18104',
  'description': 'SEX BY AGE BY COGNITIVE DIFFICULTY',
  'variables': 'http://api.census.gov/data/2021/acs/acs5/groups/B18104.json',
  'universe ': 'Civilian noninstitutionalized population 5 years and over'},
 {'name': 'B17016',
  'description': 'POVERTY STATUS IN THE PAST 12 MONTHS OF FAMILIES BY FAMILY TYPE BY WORK EXPERIENCE OF HOUSEHOLDER AND SPOUSE',
  'variables': 'http://api.census.gov/data/2021/acs/acs5/groups/B17016.json',
  'universe ': 'Families'},
 {'name': 'B18105',
  'description': 'SEX BY AGE BY AMBULATORY DIFFICULTY',
  'variables': 'http://api.census.gov/data/2021/acs/acs5/groups/B18105.json',
  'universe ': 'Civilian noninstitutionalized populat

In [6]:
c.acs5.groups_url % ("2020", "acs5")

'https://api.census.gov/data/2020/acs/acs5/groups.json'

In [ ]:
len(tables)

In [ ]:
fmt_url_old = c.acs5.groups_url % ("2020", "acs5")
fmt_url = fmt_url_old.rsplit('.', 1)[0]
fmt_url = os.path.join(fmt_url, "{}.json")

def fetch_url(url):
    print(url)
    try:
        response = requests.get(url, timeout=5.0)
    except Exception as e:
        print(f"Fail: {e}")
        return None
    return response.json()


table_options = {}
urls = [fmt_url.format(group["name"]) for group in tables]

futures = []
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = [executor.submit(fetch_url, url) for url in urls]


In [ ]:
for future in as_completed(futures):
    try:
        result = future.result()
    except:
        continue
    if result is None:
        print(f"Failed {url}")
        continue
    print(result)
    table_options.update(result["variables"])

In [ ]:
with open("census_codes.json", "w") as outfile:
    json.dump(table_options, outfile, indent=4)


In [ ]:
len(table_options.keys())

In [ ]:
c.sf1.get('NAME', geo={'for': 'tract:*',
                       'in': 'state:{} county:085'.format(states.CA.fips)})


In [ ]:
c.acs5.state_zipcode("B01001_004E", Census.ALL, 94040)

In [ ]:
c.acs5.state_zipcode("B01001_004E", states.CA.fips, 94040)

In [ ]:
result = censusgeocode.onelineaddress("2645 California Street, Mountain View, CA")

In [ ]:
result

In [ ]:
states.CA.fips


In [ ]:
c.acs5.state_county_blockgroup("B01003_001E", states.CA.fips, "085", "3", "509401")

In [ ]:
api = USCensusAPI(CENSUS_API_KEY)
api.warm_cache()
api.get_census_data("B01003_001E", "2685 California Street, Mountain View, CA")

In [8]:
field_options = c.acs5.fields()
field_options


{'ucgid': {'label': 'Uniform Census Geography Identifier clause',
  'concept': 'Census API Geography Specification',
  'predicateType': 'ucgid',
  'group': 'N/A',
  'limit': 0,
  'predicateOnly': True,
  'hasGeoCollectionSupport': True},
 'B24022_060E': {'label': 'Estimate!!Total:!!Female:!!Service occupations:!!Food preparation and serving related occupations',
  'concept': 'SEX BY OCCUPATION AND MEDIAN EARNINGS IN THE PAST 12 MONTHS (IN 2021 INFLATION-ADJUSTED DOLLARS) FOR THE FULL-TIME, YEAR-ROUND CIVILIAN EMPLOYED POPULATION 16 YEARS AND OVER',
  'predicateType': 'int',
  'group': 'B24022',
  'limit': 0,
  'attributes': 'B24022_060EA,B24022_060M,B24022_060MA'},
 'B19001B_014E': {'label': 'Estimate!!Total:!!$100,000 to $124,999',
  'concept': 'HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2021 INFLATION-ADJUSTED DOLLARS) (BLACK OR AFRICAN AMERICAN ALONE HOUSEHOLDER)',
  'predicateType': 'int',
  'group': 'B19001B',
  'limit': 0,
  'attributes': 'B19001B_014EA,B19001B_014M,B19001B_014MA

In [ ]:
with open("census_codes.json", "w") as outfile:
    json.dump(field_options, outfile, indent=4)


In [ ]:
len(field_options)

In [ ]:
g = GooglePlacesAPI(GOOGLE_API_KEY)

In [ ]:
fields = ["places.displayName","places.formattedAddress","places.priceLevel"]
g.text_search("Spicy Vegetarian Food in Sydney, Australia", fields)